## YoloV8 (detection on P0) 

In [1]:
import os
import tqdm
import mmcv
import torch
import json
import numpy as np
import pandas as pd

## Constants

In [2]:
DATASET_PATH = '/home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany'
GT_PATH = f'{DATASET_PATH}/annotations/gt.tsv'

RESULTS_PATH = '/home/ubuntu/git/B2B_ALGO/results/ALGO1'

## Data

In [3]:
gt_df = pd.read_csv(GT_PATH, sep='\t')
gt_df

,name,x_center,y_center,width,height,label,is_occluded,is_truncated,d3_separation,custom_2,back_wheel,l_label,is_not_in_lane,is_rider_on_2_wheels,is_occluded_over_80,front_wheel,r_label
0,1662979321795878.png,2171.122632,970.353636,156.774697,184.322566,TRAILER,0,0,2117.500108,1,"(0, 0)",SIDE,0,0,0,"(0, 0)",BACK
1,1662979321795878.png,1880.390755,970.000000,83.218490,99.000000,VAN_CIPV,0,0,0.000000,1,"(0, 0)",BACK,0,0,0,"(0, 0)",BACK
2,1662979321795878.png,2070.000000,994.000000,64.000000,113.500000,CAR_NLV,1,0,2075.689347,1,"(0, 0)",SIDE,0,0,0,"(0, 0)",BACK
3,1662979321795878.png,1659.000000,948.000000,36.000000,11.000000,IGNORE_VEHICLE,0,0,0.000000,1,"(0, 0)",None,0,0,0,"(0, 0)",None
4,1662979321795878.png,1590.874271,946.900594,25.344941,14.297744,IGNORE_VEHICLE,0,0,0.000000,1,"(0, 0)",None,0,0,0,"(0, 0)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96492,ab_car_munich_urban_day_002_000269.png,1445.018250,1008.892178,81.573611,44.685696,CAR,1,0,1460.167635,1,"(0, 0)",FRONT,0,0,0,"(0, 0)",SIDE
96493,ab_car_munich_urban_day_002_000269.png,2095.859645,954.738743,110.559838,155.699758,BUS,1,0,2052.522918,1,"(0, 0)",SIDE,0,0,0,"(0, 0)",BACK
96494,ab_car_munich_urban_day_002_000269.png,3482.442100,1330.664689,558.194231,508.029601,BIKE_NLV,0,0,0.000000,1,"(0, 0)",None,0,1,0,"(0, 0)",None
96495,ab_car_munich_urban_day_002_000269.png,2375.449778,1030.686752,58.180409,82.008635,BIKE,0,0,0.000000,1,"(0, 0)",None,0,1,0,"(0, 0)",None


In [4]:
paths  = f'{DATASET_PATH}/images' + '/' + np.unique(gt_df.name.values)
len(paths), paths[:3]

(4441,
 array(['/home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany/images/1662976481024636.png',
        '/home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany/images/1662976482957996.png',
        '/home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany/images/1662976486024705.png'],
       dtype=object))

## Algo

In [5]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.120 🚀 Python-3.8.16 torch-2.0.0+cu117 CUDA:0 (NVIDIA A10G, 22564MiB)
Setup complete ✅ (16 CPUs, 62.1 GB RAM, 865.6/969.3 GB disk)


In [6]:
from ultralytics import YOLO

model = YOLO('yolov8x.pt')
model.conf = 0.25
model.iou = 0.5
# prediction = model.predict(image, imgsz=image_size, show=False, save=False)

In [10]:
results = model.predict(paths[0],save=True)


image 1/1 /home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany/images/1662976481024636.png: 320x640 10 cars, 1 umbrella, 10.7ms
Speed: 1.5ms preprocess, 10.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [9]:
result = results[0]
for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Object type: car
Coordinates: [1749, 948, 1988, 1024]
Probability: 0.74
---
Object type: car
Coordinates: [1361, 950, 1637, 1069]
Probability: 0.72
---
Object type: car
Coordinates: [308, 960, 569, 1083]
Probability: 0.71
---
Object type: car
Coordinates: [489, 954, 814, 1072]
Probability: 0.55
---
Object type: umbrella
Coordinates: [40, 9, 272, 189]
Probability: 0.41
---
Object type: car
Coordinates: [725, 942, 841, 1015]
Probability: 0.4
---
Object type: car
Coordinates: [829, 951, 1177, 1058]
Probability: 0.4
---
Object type: car
Coordinates: [1571, 937, 1793, 1019]
Probability: 0.4
---
Object type: car
Coordinates: [1148, 945, 1332, 1040]
Probability: 0.4
---
Object type: car
Coordinates: [789, 960, 977, 1058]
Probability: 0.38
---
Object type: car
Coordinates: [32, 934, 286, 1085]
Probability: 0.38
---
